In [1]:
import pathlib
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import guitarpro as pygp
import glob

path_to_general_read_folder = "..\..\data\DadaGP\DadaGP-v1.1"
path_to_general_write_folder = "..\..\data\BGTG\BGTG-v0.1"

In [2]:
df_rg = pd.read_csv('../../data/df_rg_final.csv')

In [3]:
df_rg.head()

,Fichier,Dadagp_Path,File_Name,Track_Name,Partie,Instrument,Mesure,rg-estimation
0,../data/DadaGP8-gpif/ - Isolated.gpif,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...,- Isolated,Isolated\nDr. Sin,E. Ardanuy,distorted,1,2.570728e-07
1,../data/DadaGP8-gpif/ - Isolated.gpif,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...,- Isolated,Isolated\nDr. Sin,E. Ardanuy,distorted,2,4.580104e-07
2,../data/DadaGP8-gpif/ - Isolated.gpif,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...,- Isolated,Isolated\nDr. Sin,E. Ardanuy,distorted,3,6.601997e-06
3,../data/DadaGP8-gpif/ - Isolated.gpif,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...,- Isolated,Isolated\nDr. Sin,E. Ardanuy,distorted,4,4.111985e-04
4,../data/DadaGP8-gpif/ - Isolated.gpif,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...,- Isolated,Isolated\nDr. Sin,E. Ardanuy,distorted,5,4.352387e-05


In [4]:
df_inst_part = df_rg.drop_duplicates(subset=['Dadagp_Path', 'Partie'])
df_inst_part.shape, df_rg.shape

((56419, 8), (4164431, 8))

In [ ]:
total_df_track = pd.DataFrame()
i=0

# For each track we add the DadaGP_name column
for track in tqdm(df_inst_part['File_Name'].unique()):
    df_track = df_inst_part[df_inst_part['File_Name'] == track].copy() # We copy to avoid SettingWithCopyWarning
    
    dada_gp_path = df_track['Dadagp_Path'].values[0]
    gp_file = pygp.parse(dada_gp_path)
    
    # No need for this anymore normally as the path are retrieved from DadaGP
    # try:
    #     gp_file = pygp.parse(dada_gp_path)
    # except:
    #     print('Error with file:', dada_gp_path)
    #     continue
        
    inst_list = []
    for part in gp_file.tracks:
        part_name = part.name
        
        if part_name not in df_track['Partie'].values:
            continue
        inst_name = df_track[df_track['Partie'] == part_name]['Instrument'].values[0]
        inst_list.append((part_name, inst_name))
    
    # Create the DadaGP_name column filled with 'not_filled_yet'
    df_track['DadaGP_name'] = 'not_filled_yet'
    nb_clean = 0
    nb_distorted = 0
    for part_inst_tuple in inst_list:
        part = part_inst_tuple[0]
        inst = part_inst_tuple[1]
        if inst == 'clean':
            df_track.loc[df_track['Partie'] == part, 'DadaGP_name'] = 'clean' + str(nb_clean)
            nb_clean += 1
        elif inst == 'distorted':
            df_track.loc[df_track['Partie'] == part, 'DadaGP_name'] = 'distorted' + str(nb_distorted)
            nb_distorted += 1
        else:
            df_track.loc[df_track['Partie'] == part, 'DadaGP_name'] = inst
    
    # Join df_track with df_inst_part
    total_df_track = pd.concat([total_df_track, df_track])
    i+=1
    if i == 1000:
        break

df_inst_part.shape, total_df_track.shape

# The method is correct but takes a long time to compute (1h)

  0%|          | 99/24051 [00:19<1:17:42,  5.14it/s]


((56419, 8), (255, 9))

In [6]:
total_df_track.head()

,Fichier,Dadagp_Path,File_Name,Track_Name,Partie,Instrument,Mesure,rg-estimation,DadaGP_name
0,../data/DadaGP8-gpif/ - Isolated.gpif,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...,- Isolated,Isolated\nDr. Sin,E. Ardanuy,distorted,1,2.570728e-07,distorted0
131,../data/DadaGP8-gpif/ - Tudo Que Vai-Trilha 1....,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Tud...,- Tudo Que Vai,Tudo Que Vai\nCapital Inicial by Bifaum,Trilha 1,clean,2,3.238349e-05,clean0
341,../data/DadaGP8-gpif/ - Tudo Que Vai-Trilha 2....,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Tud...,- Tudo Que Vai,Tudo Que Vai\nCapital Inicial by Bifaum,Trilha 2,clean,2,9.926770e-01,clean1
386,../data/DadaGP8-gpif/1 God - Grace.gpif,..\..\data\DadaGP\DadaGP-v1.1\1\1 God\1 God - ...,1 God - Grace,Grace\n1 God,Track 1,clean,1,2.090054e-08,clean0
485,../data/DadaGP8-gpif/1 God - Loom.gpif,..\..\data\DadaGP\DadaGP-v1.1\1\1 God\1 God - ...,1 God - Loom,Loom \n1 God,Track 1,clean,1,1.952298e-07,clean0
